In [5]:
import numpy as np
import csv
import sys
from scipy.spatial.distance import cosine

In [4]:
import fasttext

#model = fasttext.train_unsupervised('data/fil9')
#model.save_model("result")

model = fasttext.load_model("result.bin")
#model.get_word_vector("the")
#model.get_nearest_neighbors('pidgey')
#model.get_analogies("psx", "sony", "nintendo")

SyntaxError: 'return' outside function (3320626788.py, line 10)

In [1]:
import fasttext

model1 = fasttext.train_unsupervised('file.txt')
model1.save_model("result_prova.bin")

model1 = fasttext.load_model("result_prova.bin")
#model1.get_word_vector("happy")
#model1.get_nearest_neighbors('jesus')
#model1.get_analogies("psx", "sony", "nintendo")

Read 0M words
Number of words:  37
Number of labels: 0
Progress: 100.0% words/sec/thread:   19192 lr:  0.000000 avg.loss:  4.117981 ETA:   0h 0m 0s
